In [1]:
# !pip install thefuzz
# !pip3 install https://github.com/explosion/spacy-models/releases/download/ru_core_news_lg-3.3.0/ru_core_news_lg-3.3.0.tar.gz
# !pip3 install https://github.com/explosion/spacy-models/releases/download/en_core_web_lg-3.3.0/en_core_web_lg-3.3.0.tar.gz
# !pip install transformers
# !pip install sentencepiece

In [1]:
from paraphraser import Paraphraser
from semantic_modification import SemChanger
import pandas as pd
from tqdm import tqdm
from multiprocessing.dummy import Pool, Queue
import shelve

df = pd.read_csv('data/mos/train.csv')
df = df.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1)
df.head(1)

In [3]:
prph = Paraphraser(device='cuda')
changer = SemChanger()

In [4]:
PROCESSES = 4
queue = Queue()

for link in range(df.shape[0]):
    queue.put(link)
    
queue.empty()

False

In [7]:
rows = {i : [] for i in range(PROCESSES)}
def process_page_wrapper(i):
    
    while not queue.empty():
        with shelve.open(f'backup_{i}.db') as db:
            num = queue.get()
            row = df.iloc[num]

            text = row['text']
            par_text = prph(text)
            sem_text = changer.change_facts(text)
            par_sem_text = changer.change_facts(par_text)
            new_row = {'art_num': i, 'orig_text': text, 'par_text': par_text, 
                           'sem_text': sem_text, 'par_sem_text': par_sem_text}
            rows[i].append(new_row)
            db[str(num)] = new_row

            with lock:
                pbar.update(1)

In [ ]:
with Pool(processes=4) as pool, tqdm(total=queue.qsize()) as pbar:
    lock = pbar.get_lock()
    pool.map(process_page_wrapper, range(pool._processes))

pool.join()

  6%|███▉                                                             | 133/2202 [30:38<6:53:21, 11.99s/it]